In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/College/6th sem/DA/DA Mini Project')

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
import Models 

Using TensorFlow backend.


In [5]:
# load packages
import pandas as pd
import gc
import tensorflow as tf
import pickle
import numpy as np
import keras
from keras import backend as K
from keras.models import load_model, Model
from keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU

from keras.utils import np_utils
import matplotlib.pyplot as plt

# set random seeds
np.random.seed(1)
# tf.random.set_seed(2)


In [6]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    dataX = dataX.swapaxes(1,2)

    return dataX, dataY

In [7]:
dec_train = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt')
dec_test3 = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt')
dec_test4 = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt')
dec_test5 = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt')
dec_test = np.hstack(( dec_test3, dec_test4, dec_test5))
del dec_test3
del dec_test4
del dec_test5
gc.collect()

# extract limit order book data from the FI-2010 dataset
train_lob = prepare_x(dec_train)
test_lob = prepare_x(dec_test)

# extract label from the FI-2010 dataset
train_label = get_label(dec_train)
test_label = get_label(dec_test)

# prepare training data. We feed past 100 observations into our algorithms and choose the prediction horizon. 
trainX_CNN, trainY_CNN = data_classification(train_lob, train_label, T=10)
trainY_CNN = trainY_CNN[:,3] - 1
trainY_CNN = np_utils.to_categorical(trainY_CNN, 3)

# prepare test data.
testX_CNN, testY_CNN = data_classification(test_lob, test_label, T=10)
testY_CNN = testY_CNN[:,3] - 1
testY_CNN = np_utils.to_categorical(testY_CNN, 3)

In [8]:
template = [[40,10], [120,5], [3,1]]

In [9]:
projection_regularizer = None
projection_constraint = keras.constraints.max_norm(3.0,axis=0)
attention_regularizer = None
attention_constraint = keras.constraints.max_norm(5.0, axis=1)
dropout = 0.1

In [10]:
model = Models.TABL(template, dropout, projection_regularizer, projection_constraint, attention_regularizer, attention_constraint)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(None, 40, 10)
(None, 120, 5)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 10)            0         
_________________________________________________________________
bl_1 (BL)                    (None, 120, 5)            5450      
_________________________________________________________________
activation_1 (Activation)    (None, 120, 5)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120, 5)            0         
_________________________________________________________________
tabl_1 (TABL)                (None, 3)                 394       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 

In [11]:
# create class weight
class_weight = {0 : 1e6/300.0,
                1 : 1e6/400.0,
                2 : 1e6/300.0}


In [12]:
# training
model.fit(trainX_CNN, trainY_CNN, batch_size=256, epochs=70, class_weight=class_weight)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/70
254741/254741 [==============================] - 14s 54us/step - loss: 1666.3158 - acc: 0.7309
Epoch 2/70
254741/254741 [==============================] - 14s 53us/step - loss: 1499.8679 - acc: 0.7665
Epoch 3/70
254741/254741 [==============================] - 13s 52us/step - loss: 1454.8358 - acc: 0.7750
Epoch 4/70
254741/254741 [==============================] - 13s 53us/step - loss: 1440.0639 - acc: 0.7777
Epoch 5/70
254741/254741 [==============================] - 13s 53us/step - loss: 1438.0867 - acc: 0.7782
Epoch 6/70
254741/254741 [==============================] - 13s 53us/step - loss: 1427.2572 - acc: 0.7796
Epoch 7/70
254741/254741 [==============================] - 13s 51us/step - loss: 1422.3537 - acc: 0.7815
Epoch 8/70
254741/254741 [==============================] - 13s 51us/step - loss: 1420.4125 - acc: 0.7812
Epoch 9/70
254741/254741 [==============================]

In [15]:
from sklearn.metrics import classification_report

y_pred = model.predict(testX_CNN, batch_size=256, verbose=2)
y_pred_bool = np.argmax(y_pred, axis=1)

round_testy = np.argmax(testY_CNN, axis=1)

print(classification_report(round_testy, y_pred_bool))


              precision    recall  f1-score   support

           0       0.64      0.67      0.66     38464
           1       0.83      0.72      0.77     66002
           2       0.59      0.71      0.64     35112

    accuracy                           0.70    139578
   macro avg       0.69      0.70      0.69    139578
weighted avg       0.72      0.70      0.71    139578

